In [2]:
import pandas as pd

#### DDA 선택 기준
- 첫번째: 분석 제공 대상 선정 _Columnn 선택 기준 달라짐
     - 환자 or 보호자
     - 의사
     - 병원 운영 관리자
     - 연구자
     - 보험회사

In [40]:
df_ROS = pd.read_csv('../../datasets/RecurrenceOfSurgery.csv')
df_ROS.head(4)

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0
2,2,3PT,53.0,3,NaN,0.0,0,0,1,0,...,0,67.5,27.3,10.2,0,-1.77,1749.0,4,Down,0
3,3,4PT,54.7,4,18.0,0.0,0,1,0,0,...,0,49.2,18.7,19.9,0,-1.58,1796.2,4,Down,0


In [6]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

### 데이터 핸들링
- 정렬 (column으로 setting 후 정렬하는데 잘 안씀.)
    - pd.sort_values(by=['column'], ascending=[False or True])
- 추출 
    - 열 단위 추출: df[['a','b','c'...]] column list로 한다. 
    - 행 단위 추출: 주로 iloc[시작:끝, 시작:끝] (location)index로 움직임 (선행동작필) 
        - ex) .iloc[:3] #[:3] =0번부터~2번/ .iloc[-4:] = 뒤에서 4번째행까지만  - & head(앞에서) or tail(뒤에서)
- 필터 (특정한것 추출 + sum 같은 계산할때 필요한것 추출 ) pd.loc[conditions]
- 요약 
    - df_saledays_dropcol.pivot_table(index='반품여부', values='가격', aggfunc=np.std)
    - index : group by
    - value : 대상 column
    - aggfunc : 대상 산술
    

### 환자 대상 분석 필요
0. 변수 구조 확인 (.columns)
1. 목표변수 설정 (y축)
    - → column 확인하기 
    - ⇒ '재발여부' 선택
2. 설명변수 설정 (x축)
    - '성별','신장', '심혈관질환','체중','흡연여부','직업','혈액형' 후보 선택
    - ⇒ 설명변수 여러개 중에 추출을 한다 (열 단위 추출 진행 (이때 목표변수도 같이 넣고 추출))



In [41]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

df_ROS.columns는 instance라 () 필요없음. 
<환자가 궁금할거같은 column_재발여부가 궁금>
-재발여부 = 목표변수 

In [42]:
# '재발여부','성별','신장', '심혈관질환','체중','흡연여부','직업','혈액형'
# 열 단위 추출 방법_'하나'만 넣으면 series로 인식함. 고로, 이중괄호로 진행 
# df_ROS[['재발여부','성별','신장', '심혈관질환','체중','흡연여부','직업','혈액형']]
df_ROS_forpatient = df_ROS[['재발여부','성별','신장', '심혈관질환','체중','흡연여부','직업','혈액형']]
# df_ROS_forpatient.head(4)
# df_ROS_forpatient.tail(3)
df_ROS_forpatient.tail(3)

,재발여부,성별,신장,심혈관질환,체중,흡연여부,직업,혈액형
1891,0,1,167,0,70.0,0,무직,RH+O
1892,0,1,177,0,77.0,0,사무직,RH+A
1893,0,2,168,0,49.0,0,사무직,RH+AB


In [43]:
# 행단위 추출 _.iloc[:, :],.head(), .tail()
df_ROS_forpatient.iloc[:3] #[:3] =0번부터~2번

,재발여부,성별,신장,심혈관질환,체중,흡연여부,직업,혈액형
0,0,2,163,0,60.3,0,자영업,RH+A
1,0,1,171,0,71.7,0,운동선수,RH+A
2,0,1,178,0,77.1,0,특수전문직,RH+B


In [44]:
df_ROS_forpatient.iloc[-4:] #.iloc[-4:] = 뒤에서 4번째행까지만 

,재발여부,성별,신장,심혈관질환,체중,흡연여부,직업,혈액형
1890,0,2,157,0,59.0,0,사무직,RH+B
1891,0,1,167,0,70.0,0,무직,RH+O
1892,0,1,177,0,77.0,0,사무직,RH+A
1893,0,2,168,0,49.0,0,사무직,RH+AB


#### index 관련 사항
df_ROS.columns에서 보이는 index 부분을 column으로 대체 가능
- df_ROS.set_index('환자ID')


In [45]:
df_ROS.set_index('환자ID') # uniqueID 경우

,Unnamed: 0,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,말초동맥질환여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
환자ID,,,,,,,,,,,,,,,,,,,,,
1PT,0,22.8,3,51.0,0.0,0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.50,4,Down,0
2PT,1,44.9,4,26.0,0.0,0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.10,4,Up,0
3PT,2,53.0,3,NaN,0.0,0,0,1,0,0,...,0,67.5,27.3,10.2,0,-1.77,1749.00,4,Down,0
4PT,3,54.7,4,18.0,0.0,0,1,0,0,0,...,0,49.2,18.7,19.9,0,-1.58,1796.20,4,Down,0
5PT,4,53.8,4,NaN,0.0,0,1,0,0,0,...,0,58.8,14.7,5.2,0,-0.79,2021.48,4,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890PT,1889,44.0,1,NaN,0.0,0,0,0,0,0,...,2,59.5,23.0,21.8,0,NaN,2652.20,5,Middle,0
1891PT,1890,30.8,2,NaN,0.0,0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.80,4,Down,0
1892PT,1891,36.7,3,NaN,0.0,0,0,0,0,1,...,0,44.6,15.0,17.4,1,NaN,2624.40,4,Down,0


In [46]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [47]:
df_ROS.iloc[-4:]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
1890,1890,1891PT,30.8,2,NaN,0.0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.8,4,Down,0
1891,1891,1892PT,36.7,3,NaN,0.0,0,0,0,0,...,0,44.6,15.0,17.4,1,NaN,2624.4,4,Down,0
1892,1892,1893PT,32.5,2,NaN,0.0,0,0,0,0,...,2,32.2,11.1,17.7,0,NaN,1972.5,5,Down,0
1893,1893,1894PT,49.5,2,NaN,0.0,0,0,0,0,...,0,38.9,6.8,27.8,0,NaN,1574.4,5,Middle,0


In [48]:
### 정렬 _ sort_values(by=[두개이상]
df_ROS_forpatient.sort_values(by=['신장'])

,재발여부,성별,신장,심혈관질환,체중,흡연여부,직업,혈액형
1763,0,2,140,1,35.0,0,무직,RH+O
896,0,2,140,0,35.0,0,NaN,RH+A
348,0,2,142,0,52.9,0,자영업,RH+A
70,0,2,143,0,53.3,0,사무직,RH+O
974,0,2,143,0,56.0,0,무직,RH+O
...,...,...,...,...,...,...,...,...
202,0,1,189,0,85.3,0,군인,RH+B
464,0,1,189,0,85.9,0,자영업,RH+A
1227,0,1,190,0,74.0,0,자영업,RH+O
1246,0,1,191,0,114.0,0,학생,RH+O


In [49]:
df_ROS_forpatient.sort_values(by=['신장','체중'], ascending=[False, False])
# ascending=[False] False는 descending?! == orderby 정렬된 범위내에서 2번째가 됨. 


,재발여부,성별,신장,심혈관질환,체중,흡연여부,직업,혈액형
1579,1,1,204,0,103.0,0,교사,RH+O
1246,0,1,191,0,114.0,0,학생,RH+O
1227,0,1,190,0,74.0,0,자영업,RH+O
464,0,1,189,0,85.9,0,자영업,RH+A
202,0,1,189,0,85.3,0,군인,RH+B
...,...,...,...,...,...,...,...,...
974,0,2,143,0,56.0,0,무직,RH+O
70,0,2,143,0,53.3,0,사무직,RH+O
348,0,2,142,0,52.9,0,자영업,RH+A
896,0,2,140,0,35.0,0,NaN,RH+A


In [51]:
## simple quest
## 정렬_혈액형 ascending, 직업 descending
## 추출_혈액형, 직업, 성별
# df_ROS_forpatient.sort_values(by=['신장','체중'], ascending=[False, False])
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [56]:
# 추출: 혈액형, 직업, 성별
df_ROS_BJG = df_ROS[['혈액형','직업','성별']]
df_ROS_BJG.head(5)

,혈액형,직업,성별
0,RH+A,자영업,2
1,RH+A,운동선수,1
2,RH+B,특수전문직,1
3,RH+O,주부,1
4,RH+A,특수전문직,1


In [58]:
# 정렬: 혈액형 asc, 직업 des
df_ROS_BJGsort = df_ROS_BJG.sort_values(by=['혈액형','직업'], ascending=[True, False])
df_ROS_BJGsort.iloc[:5]

,혈액형,직업,성별
60,RH+A,학생,1
176,RH+A,학생,1
184,RH+A,학생,2
224,RH+A,학생,2
272,RH+A,학생,1


In [106]:
## simple quest 2
# 직업군 상위 2위 속하는 항목(value로)_빈도수 나온 상태에서 dataframe으로 바꾼뒤 iloc[:2]
# → value_count, dataFrame, iloc[:2] 잘라
# 체중이 70 초과, 신장이 170 초과, 흡연여부 0 전부 & 조건
# 해당 레코드 개수 표현
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [107]:
df_ROSJOB = df_ROS['직업'].value_counts()
df_ROSJOB_list=(pd.DataFrame(df_ROSJOB))
df_ROSJOB_list.iloc[:2]
# 사무직, 자영업

,직업
사무직,449
자영업,171


In [108]:
df_ROS_DF = pd.DataFrame(df_ROS)
condition = ( (df_ROS_DF['직업'] == '사무직') | (df_ROS_DF['직업'] == '자영업') )
df_ROS_DF_top2=df_ROS_DF.loc[condition]

In [115]:
# 체중이 70 초과, 신장이 170 초과, 흡연여부 0
condition = (df_ROS_DF_top2['신장'] > 170 ) & (df_ROS_DF_top2['체중'] > 70 ) & (df_ROS_DF_top2['흡연여부'] == 0)
df_ROS_DF_top2_fin = df_ROS_DF_top2.loc[condition]
df_ROS_DF_top2_fin

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
40,40,41PT,42.7,5,NaN,0.0,0,0,0,0,...,2,35.8,16.1,14.6,0,-1.98,2040.7,4,Middle,0
53,53,54PT,38.6,3,NaN,0.0,0,0,0,0,...,0,33.6,10.6,7.0,0,-1.10,2099.3,4,Middle,0
72,72,73PT,34.6,2,NaN,0.0,0,0,0,0,...,0,38.4,16.2,16.1,0,-2.34,2387.1,3,Down,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,1830,1831PT,47.9,1,NaN,0.0,0,0,0,0,...,0,56.3,19.7,26.4,0,NaN,2310.0,4,Middle,0
1841,1841,1842PT,33.1,1,NaN,0.0,0,0,0,0,...,0,39.2,14.3,9.1,0,NaN,2067.0,4,Middle,0
1855,1855,1856PT,55.6,2,NaN,0.0,0,0,0,0,...,0,58.8,15.8,18.6,0,NaN,2149.3,4,Middle,0
1871,1871,1872PT,51.1,1,NaN,0.0,0,0,0,0,...,0,42.1,7.0,24.9,0,NaN,2102.3,4,Extremely down,0


,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
직업,,,,,,,,,,,,,,,,,,,,,
사무직,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
사무직,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
사무직,40,41PT,42.7,5,NaN,0.0,0,0,0,0,...,2,35.8,16.1,14.6,0,-1.98,2040.7,4,Middle,0
사무직,53,54PT,38.6,3,NaN,0.0,0,0,0,0,...,0,33.6,10.6,7.0,0,-1.10,2099.3,4,Middle,0
자영업,72,73PT,34.6,2,NaN,0.0,0,0,0,0,...,0,38.4,16.2,16.1,0,-2.34,2387.1,3,Down,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
자영업,1830,1831PT,47.9,1,NaN,0.0,0,0,0,0,...,0,56.3,19.7,26.4,0,NaN,2310.0,4,Middle,0
자영업,1841,1842PT,33.1,1,NaN,0.0,0,0,0,0,...,0,39.2,14.3,9.1,0,NaN,2067.0,4,Middle,0
사무직,1855,1856PT,55.6,2,NaN,0.0,0,0,0,0,...,0,58.8,15.8,18.6,0,NaN,2149.3,4,Middle,0


In [100]:
df_ROS_DF_top2_asc.set_index('직업')

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
직업,,,,,,,,,,,,,,,,,,,,,
사무직,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.20,4,Down,0
사무직,1164,1165PT,32.1,1,68.0,0.0,0,0,2,0,...,0,69.5,36.7,3.5,0,NaN,2278.40,45,Middle,0
사무직,1184,1185PT,46.1,4,NaN,0.0,0,0,0,1,...,0,37.1,1.6,16.6,0,NaN,2287.70,4,Middle,0
사무직,1197,1198PT,51.7,1,47.0,0.0,0,0,2,0,...,0,57.6,18.6,12.8,0,NaN,2109.70,4,Middle,0
사무직,1216,1217PT,53.1,3,12.0,0.0,0,0,3,0,...,0,50.5,13.2,12.7,0,1.40,2109.50,4,Middle,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
자영업,316,317PT,58.4,1,NaN,0.0,0,0,0,0,...,0,57.5,18.4,14.3,0,-0.83,1473.30,4,Down,0
자영업,1498,1499PT,44.9,3,NaN,0.0,0,0,0,0,...,0,34.2,-0.6,17.8,0,NaN,1727.00,5,Middle,0
자영업,1524,1525PT,35.0,2,NaN,0.0,0,0,0,0,...,0,55.9,21.7,16.5,0,NaN,1789.00,4,Middle,0


1. 직업top2 640개
2. top2에서 모든 조건에 부합하는 레코드만 출력. 164개

value_count를 변수로 담아???
- .value_counts().index[0 or 1]
- .value_counts().iloc[:2].value
- .value_counts().iloc[:2].index

### Quest 3
- quest 2 집합 구성
- 직업별 성별
- 직업별 혈액형 


In [155]:
# QUEST 2의 최종 data
df_ROS_DF_top2_fin

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
40,40,41PT,42.7,5,NaN,0.0,0,0,0,0,...,2,35.8,16.1,14.6,0,-1.98,2040.7,4,Middle,0
53,53,54PT,38.6,3,NaN,0.0,0,0,0,0,...,0,33.6,10.6,7.0,0,-1.10,2099.3,4,Middle,0
72,72,73PT,34.6,2,NaN,0.0,0,0,0,0,...,0,38.4,16.2,16.1,0,-2.34,2387.1,3,Down,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,1830,1831PT,47.9,1,NaN,0.0,0,0,0,0,...,0,56.3,19.7,26.4,0,NaN,2310.0,4,Middle,0
1841,1841,1842PT,33.1,1,NaN,0.0,0,0,0,0,...,0,39.2,14.3,9.1,0,NaN,2067.0,4,Middle,0
1855,1855,1856PT,55.6,2,NaN,0.0,0,0,0,0,...,0,58.8,15.8,18.6,0,NaN,2149.3,4,Middle,0
1871,1871,1872PT,51.1,1,NaN,0.0,0,0,0,0,...,0,42.1,7.0,24.9,0,NaN,2102.3,4,Extremely down,0


- 요약 
    - df_saledays_dropcol.pivot_table(index='반품여부', values='가격', aggfunc=np.std)
    - index : group by
    - value : 대상 column
    - aggfunc : 대상 산술

In [129]:
import numpy as np

In [171]:
df_ROS_DF_top2_fin.pivot_table(index=['성별'],columns=['직업'], aggfunc='size')

직업,사무직,자영업
성별,,
1,121.0,39.0
2,4.0,NaN


In [172]:
df_ROS_DF_top2_fin.pivot_table(index=['혈액형'],columns=['직업'], aggfunc='size')

직업,사무직,자영업
혈액형,,
RH+A,50,21
RH+AB,13,3
RH+B,24,8
RH+O,38,7


In [207]:
df_ROS_DF_top2_fin.pivot_table(index=['성별','직업'],columns=['직업'], aggfunc=np.size)

Instability       Large Lymphocyte       Location of herniation        \
직업             사무직   자영업              사무직   자영업                    사무직   자영업   
성별 직업                                                                          
1  사무직       121.0   NaN            121.0   NaN                  121.0   NaN   
   자영업         NaN  39.0              NaN  39.0                    NaN  39.0   
2  사무직         4.0   NaN              4.0   NaN                    4.0   NaN   

       MF + ES       Modic change        ... 혈전합병증여부         환자ID        \
직업         사무직   자영업          사무직   자영업  ...     사무직   자영업    사무직   자영업   
성별 직업                                    ...                              
1  사무직   121.0   NaN        121.0   NaN  ...   121.0   NaN  121.0   NaN   
   자영업     NaN  39.0          NaN  39.0  ...     NaN  39.0    NaN  39.0   
2  사무직     4.0   NaN          4.0   NaN  ...     4.0   NaN    4.0   NaN   

       환자통증정도       후방디스크높이(mm)         흡연여부        
직업        사무직   자영업         사무직   자영업    사무직   자영업  
성별 직업                                               
1  사무직  121.0   NaN       121.0   NaN  121.0   NaN  
   자영업    NaN  39.0         NaN  39.0    NaN  39.0  
2  사무직    4.0   NaN         4.0   NaN    4.0   NaN  

[3 rows x 100 columns]